In [ ]:
file_path = "Bangla-NER-Splitted-Dataset.json"

In [ ]:
import json
  
# Opening JSON file
f = open(file_path, mode="r", encoding="utf-8")
  
# returns JSON object as 
# a dictionary
json_data = json.load(f)

In [ ]:
json_data.keys()

In [ ]:
train_json_data = json_data['train']
val_json_data = json_data['validation']
test_json_data = json_data['test']

In [ ]:
import pandas as pd
import re
import numpy as np

train_df = pd.DataFrame(columns=['ner_tags', 'tokens'])
val_df = pd.DataFrame(columns=['ner_tags', 'tokens'])
test_df = pd.DataFrame(columns=['ner_tags', 'tokens'])

In [ ]:
def read_json_file(json_file, df):
    token_docs = []
    tag_docs = []

    for idx,doc in enumerate(json_file):
        tokens = doc['sentence']
        tags = doc['iob_tags']
        token_docs.append(tokens)
        tag_docs.append(tags)
        df.loc[idx] = pd.Series({'ner_tags':tags, 'tokens':tokens})
    
    return df, token_docs, tag_docs

In [ ]:
train_df,_,tag_docs = read_json_file(train_json_data, train_df)

In [ ]:
val_df,_,_ = read_json_file(val_json_data, val_df)

In [ ]:
test_df,_,_ = read_json_file(test_json_data, val_df)

In [ ]:
train_df[:1]

In [ ]:
val_df[:1]

In [ ]:
test_df[:1]

In [ ]:
from datasets import Dataset, DatasetDict

train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
test_ds = Dataset.from_pandas(test_df)

In [ ]:
datasets = DatasetDict()
datasets['train'] = train_ds
datasets['validation'] = val_ds
datasets['test'] = test_ds

In [ ]:
datasets["train"][3]["tokens"]

In [ ]:
datasets["train"][3]["ner_tags"]

In [ ]:
# create set from list
expanded_tag_docs = []

for tags in tag_docs:
    for tag in tags:
        expanded_tag_docs.append(tag)

In [ ]:
unique_tags = set(expanded_tag_docs)
unique_tags

In [ ]:
def assign_label(examples):
    mapping = {'B-LOC':0, 'B-OBJ':1, 'B-ORG':2, 'B-PER':3, 'I-LOC':4, 'I-OBJ':5, 'I-ORG':6, 'I-PER':7, 'O':8}
    ner_labels = []
    for example in examples["ner_tags"]:
        ner_labels.append(mapping[example])
    examples["ner_labels"] = ner_labels
    return examples

In [ ]:
datasets = datasets.map(assign_label)

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "../Bengali Pretraining/models/unigram/unigram-long-text"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# num_added_tokens = tokenizer.add_tokens(["-","’","০","১","২","৩","৪","৫","৬","৭","৮","৯"])

In [ ]:
inputs = tokenizer(datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

In [ ]:
inputs.word_ids()

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
labels = datasets["train"][0]["ner_labels"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

In [ ]:
datasets["train"][1]

In [ ]:
mapping = {'B-LOC':0, 'B-OBJ':1, 'B-ORG':2, 'B-PER':3, 'I-LOC':4, 'I-OBJ':5, 'I-ORG':6, 'I-PER':7, 'O':8}
label_names = list()
for k, v in mapping.items():
    label_names.append(k)

label_names

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, max_length=128
    )
    all_labels = examples["ner_labels"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=datasets["train"].column_names,
)

In [ ]:
tokenized_datasets

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

In [ ]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

In [ ]:
import evaluate

metric = evaluate.load("seqeval")

In [ ]:
labels = datasets["train"][0]["ner_labels"]
labels = [label_names[i] for i in labels]
labels

In [ ]:
predictions = labels.copy()
predictions[2] = 'B-PER'
metric.compute(predictions=[predictions], references=[labels])

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
id2label

In [ ]:
label2id

In [ ]:
from transformers import set_seed
set_seed(42)

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

In [ ]:
# model.resize_token_embeddings(len(tokenizer))

In [ ]:
model.config.num_labels

In [ ]:
#  disable weights and biases logging
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import TrainingArguments
# batch_size = 16
batch_size = 32

args = TrainingArguments(
    "models/ner",
    report_to = None,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    #save_strategy="epoch",
    learning_rate=2e-5,
    # learning_rate=3e-5,
    #num_train_epochs=3,
    num_train_epochs=4,
    weight_decay=0.01,
    warmup_ratio=0.06,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    metric_for_best_model = 'f1',
    load_best_model_at_end=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [ ]:
import transformers
trainer.remove_callback(transformers.integrations.TensorBoardCallback)

In [ ]:
trainer.evaluate()

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
# trainer.args.num_train_epochs = trainer.args.num_train_epochs + 1
# trainer.train("models/ner/checkpoint-2005")

In [ ]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer.evaluate(tokenized_datasets["test"])